# Deploy Model to online endpoint with CLI V2

- Create the 3 scripts required for Model Deployment through the CLI V2 SDK


In [12]:
import os

# Create a folder for the experiment files
script_folder = 'CLI_V2_ManagedOnlineEndpoint'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

CLI_V2_ManagedOnlineEndpoint folder created


In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"oneline endpoint": "titanic"},
)

In [6]:
%%writefile $script_folder/endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: titanic-managed-online-endpoint
description: "CLI V2 titanic online endpoint for mlflow model"
auth_mode: key
tags : {"CLIV2": "titanic"}

Writing CLI_V2_ManagedOnlineEndpoint/endpoint.yml


In [ ]:
%%writefile $script_folder/deployment.yml

$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: titanic-managed-online-endpoint
model: azureml:chapter6_titanic_model:1
code_configuration:
  code: 
    local_path: .
  scoring_script: score.py
environment: azureml:job_base_env:1
instance_type: Standard_F2s_v2
instance_count: 1

In [ ]:
%%writefile $script_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'titanic_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result